# Software Architecture

As a former software architect, I viewed my role as ensuring these things:

 - the software supported the things we needed it to do
 - constraints and conflicts among the software requirements were evident, as were the decisions made in resolving them
 - the team (including new joiners) understood the design choices, motivations behind them, and situations in which we'd review them
 - the software could easily be extended to support things we were planning

It's this last one that's the most contentious. Software engineers are rightly concerned that if [You Aren't Gonna Need It](http://c2.com/xp/YouArentGonnaNeedIt.html) (YAGNI), it's a waste of time building it (not just time wasted now, but later, in maintaining the unused code). Practices like Test-Driven Development enforce YAGNI by requiring that the only code you write is code you have tests for. Presumably, you have tests because you derived them from the requirements, and conversely if it's not required then it's not tested.

But knowing about YAGNI can lead engineers to avoid _planning_ for the future, even for things that are near certain to come about. If you know that in six months you'll get the grant to work on integrating with some popular Java packages, it makes very little sense to start working in C# now based on the idea that "You Ain't Gonna Need" JVM interoperability. If a change of policy coming into force next year means that all participant data must be stored onsite, putting your database into a public cloud today means guaranteed work - up against a deadline - later.

## C4 Model

Software Architect and consultant Simon Brown likens the architecture documentation for a software project to a series of maps: at different scales, the amount of information you show or hide is different, and the things that you call attention to change, too. He introduced the [C4 Model](https://c4model.com/) for visualising software architecture, which defines four (unsurprisingly) levels of detail.

1. _Context_ diagrams tell people a lot about the outside of a system, and nothing about the inside. The whole system is drawn as a box, with links to the people and other systems that interface with it. The context diagram calls out constraints like what data goes where and who needs to use the system.
2. _Container_ diagrams begin to break the application up into distinct high-level components, which can be physical (different parts are deployed to different servers, or into a user's web browser, for example) or logical (a web application has a "storage" service and a "reporting" service even if they're deployed at the same place, for example). At this level, it's possible to see how the functionality of the system is distributed across different parts, and how they work in concert to achieve the system's goals.
3. _Component_ diagrams dive into a particular high-level component to show its detailed design. Whether it is built with Object-Oriented or Functional Programming principles, the interfaces it exposes to communicate externally, and the modules needed within the component, can be shown in the component diagram.
4. _Code_ diagrams blah blah

In practice, I have found that context and container diagrams are very useful to orient team members, and the context diagram can even become part of the marketing or conference collateral for talking about the system. Whether a team needs component or code diagrams, and whether they are long-lived or prepared _ad hoc_ when needed, seems to be much more flexible. Some teams abhor detailed design diagrams on the basis that they repeat a lot of information in the code and are unlikely to be kept up to date.

The idea of keeping diagrams in sync with the code brings up notions of CASE (computer-assisted software engineering) tools and round-trip diagram generation. These tools are not used much in practice, for the same reason that an aerial photo of a terrain is not a good substitute for a map. The story you tell with a diagram (the 1,000 words it's standing in for) is supported by the information you choose to bring forward or to hide on the diagram. Automated tools aren't going to provide the level of discernment needed to select the relevant details, and hide the irrelevant. And a diagram that can be round-trip engineered to and from the code necessarily contains all of the same information as the code, rendering it useless as a diagram.

## Architecture Decisions

### High-level Architecture

### Technology Choices

### Build or Buy

## Walking Skeletons

## Architecture Astronauts